**데이터 관련 클래스**

In [2]:
import numpy as np

In [3]:
class Data:
    def __init__(self, mu, sigma, ni_D):
        self.real_sample = lambda n_batch: np.random.normal(mu, sigma, (n_batch, ni_D))
        self.in_sample = lambda n_batch: np.random.normal((n_batch, ni_D))

**머신 구현하기**

In [4]:
%matplotlib inline

In [6]:
import matplotlib.pyplot as plt

In [7]:
class Machine:
    def __init__(self, n_batch=10, ni_D=100):
        data_mean = 4
        data_stddev = 1.25

        self.n_iter_D = 1
        self.n_iter_G = 1

        self.data = Data(data_mean, data_stddev, ni_D)
        self.gan = GAN(ni_D=ni_D, nh_D = 50, nh_G=50)

        self.n_batch = n_batch
        self.ni_D = ni_D

    def train_D(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data

        # Real Data (lambda)
        Real = data.real_sample(n_batch)

        # Generated Data (lambda)
        Z = data.in_sample(n_batch)
        Gen = gan.G.prdict(Z)

        gan.D.trainable = True
        gan.D_train_on_batch(Real, Gen)

    def train_GD(self):
        gan = self.gan
        n_batch = self.n_batch
        data = self.data

        Z = data.in_sample(n_batch)

        gan.D.trainable = False
        gan.GD_train_on_batch(Z)

    def train_each(self):
        for it in range(self.n_iter_D):
            self.train_D()
        for it in range(self.n_iter_G):
            self.train_GD()

    def train(self, epochs):
        for epoch in range(epochs):
            self.train_each()

    def test(self, n_test):
        gan = self.gan
        data = self.data

        Z = data.in_sample(n_test)
        Gen = gan.G.predict(Z)
        return Gen, Z

    def show_hist(self, Real, Gen, Z):
        plt.hist(Real.reshape(-1), histtype='step', label='Real')
        plt.hist(Gen.reshape(-1), histtype='step', label='Generated')
        plt.hist(Z.reshape(-1), histtype='step', label='Input')
        plt.legend(loc=0)

    def test_and_show(self, n_test):
        data = self.data
        Gen, Z = self.test(n_test)
        Real = data.real_sample(n_test)

        self.show_hist(Real, Gen, Z)
        Machine.print_stat(Real, Gen)

    def run_epochs(self, epochs, n_test):
        self.train(epochs)
        self.test_and_show(n_test)

    def run(self, n_repeat=200, n_show=200, n_test=100):
        for ii in range(n_repeat):
            print('Stage', ii, '(Epoch: {})'.format(ii * n_show))
            self.run_epochs(n_show, n_test)
            plt.show()

    @staticmethod
    def print_stat(Real, Gen):
        def stat(d):
            return (np.mean(d), np.std(d))

        print('Mean and Std of Real:', stat(Real))
        print('Mean and Std of Gen:', stat(Gen))

**GAN 모델링**

In [9]:
from keras import backend as K
from keras import layers, models, optimizers

In [11]:
def add_decorate(x):
    m = K.mean(x, axis=-1, keepdims=True)
    d = K.square(x - m)

    return K.concatenate([x, d], axis=-1)

def add_decorate_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 2

    shape[1] *= 2
    return tuple(shape)

In [12]:
lr = 2e-4
adam = optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999)

In [13]:
def model_compile(model):
    model.compile(loss='binary_crossentropy', optimizers=adam, metrics=['accuracy'])

In [15]:
class GAN:
    def __init__(self, ni_D, nh_D, nh_G):
        self.ni_D = ni_D
        self.nh_D = nh_D
        self.nh_G = nh_G

        self.D = self.gen_D()
        self.G = self.gen_G()
        self.GD = self.make_GD()

    def gen_D(self):
        ni_D = self.ni_D
        nh_D = self.nh_D

        D = models.Sequential()
        D.add(
            layers.Lambda(
                add_decorate, output_shape=add_decorate_shape, input_shape=(ni_D, )
            )
        )
        D.add(
            layers.Dense(nh_D, activation='relu')
        )
        D.add(
            layers.Dense(nh_D, activation='relu')
        )
        D.add(
            layers.Dense(1, activation='sigmoid')
        )

        model_compile(D)

        return D

    def gen_G(self):
        ni_D = self.ni_D
        nh_G = self.nh_G

        G = models.Sequential()
        G.add(
            layers.Reshape((ni_D, 1), input_shape=(ni_D, ))
        )
        G.add(
            layers.Conv1D(nh_G, 1, activation='relu')
        )
        G.add(
            layers.Conv1D(nh_G, 1, activation='sigmoid')
        )
        G.add(
            layers.Conv1D(1, 1)
        )
        G.add(
            layers.Flatten()
        )

        model_compile(G)
        return G

    def make_GD(self):
        G, D = self.G, self.D
        GD = models.Sequential()
        GD.add(G)
        GD.add(D)

        D.trainable = False
        model_compile(GD)
        D.trainable = True
        return GD

    def D_train_on_batch(self, Real, Gen):
        D = self.D
        X = np.concatenate([Real, Gen], axis=0)
        y = np.array([1] * Real.shape[0] + [0] * Gen.shape[0])
        D.train_on_batch(X, y)

    def GD_train_on_batch(self, Z):
        GD = self.GD
        y = np.array([1] * Z.shape[0])
        GD.train_on_batch(Z, y)

In [17]:
def main():
    machine = Machine(n_batch=10, ni_D=100)
    machine.run(n_repeat=200, n_show=200, n_test=100)

In [18]:
main()

TypeError: Invalid keyword argument(s) in `compile`: {'optimizers'}